In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!sudo apt-get update

In [ ]:
!nvidia-smi

In [ ]:
!pip install thop
!unzip -qq '/content/gdrive/My Drive/pranet/data/Kvasir_fold_new.zip'

In [ ]:
from lib.PraNet_Res2Net import PraNet, PraNetAG, PraNetDGv2,PraNetDGv3, PraNetGALD, PraNetv12
import os

lr = 1e-4
batchsize = 16
trainsize_init = 352
clip = 0.5
decay_rate = 0.1
decay_epoch = 50
import timeit
start_from = 0
name = [[1,2,3,4], [0,2,3,4], [0,1,3,4], [0,1,2,4], [0,1,2,3]]
start = timeit.default_timer()

# for i in range(4,5):
v = 12
i = 1
train_save = 'PraNetv{}_Res2Net_kfold'.format(v)
save_path = 'snapshots/{}/'.format(train_save)
log_file = 'PraNetv{}_Res2Net_fold{}.log'.format(v,i)
Log.init(
    log_level="info",
    log_file=os.path.join(save_path, log_file),
    log_format="%(asctime)s %(levelname)-7s %(message)s",
    rewrite=False,
    stdout_level="info"
)
# ---- build models ----
# torch.cuda.set_device(0)  # set your gpu device
model = PraNetv12().cuda()
if start_from != 0: 
  restore_from = "./snapshots/PraNetv{}_Res2Net_kfold/PraNetGALD-fold{}-{}.pth".format(v,i,start_from)
  saved_state_dict = torch.load(restore_from)["model_state_dict"]
  lr = torch.load(restore_from)["lr"]

  # new_params = model.state_dict().copy()
  # for i in saved_state_dict:
  #     i_parts = i.split('.')
  #     if not i_parts[0] == 'fc':
  #         new_params['.'.join(i_parts[0:])] = saved_state_dict[i]
  model.load_state_dict(saved_state_dict, strict=False)





train1 = 'fold_' + str(name[i][0])
train2 = 'fold_' + str(name[i][1])
train3 = 'fold_' + str(name[i][2])
train4 = 'fold_' + str(name[i][3])
test_fold = 'fold' + str(i)
train_img_paths =[]
train_mask_paths = []
train_img_path_1 = glob('/content/Kvasir_fold_new/' + train1 + "/images/*")
train_img_paths.extend(train_img_path_1)

train_img_path_2 = glob('/content/Kvasir_fold_new/' + train2 + "/images/*")
train_img_paths.extend(train_img_path_2)

train_img_path_3 = glob('/content/Kvasir_fold_new/' + train3 + "/images/*")
train_img_paths.extend(train_img_path_3)

train_img_path_4 = glob('/content/Kvasir_fold_new/' + train4 + "/images/*")
train_img_paths.extend(train_img_path_4)


train_mask_path_1 = glob('/content/Kvasir_fold_new/' + train1 + "/masks/*")
train_mask_paths.extend(train_mask_path_1)

train_mask_path_2 = glob('/content/Kvasir_fold_new/' + train2 + "/masks/*")
train_mask_paths.extend(train_mask_path_2)

train_mask_path_3 = glob('/content/Kvasir_fold_new/' + train3 + "/masks/*")
train_mask_paths.extend(train_mask_path_3)

train_mask_path_4 = glob('/content/Kvasir_fold_new/' + train4 + "/masks/*")
train_mask_paths.extend(train_mask_path_4)

train_img_paths.sort()
train_mask_paths.sort()

train_transform = Compose([
        transforms.RandomRotate90(),
        transforms.Flip(),
        transforms.HueSaturationValue(),
        transforms.RandomBrightnessContrast(),
        transforms.Transpose(),
        OneOf([
          transforms.RandomCrop(220,220, p=0.5),
          transforms.CenterCrop(220,220, p=0.5)
        ], p=0.5),
        transforms.Resize(352,352)
    ])

train_dataset = Dataset(train_img_paths, train_mask_paths, transform=train_transform)

# for id in range(0, 2):
#   image, mask = train_dataset[id] # get some sample
#   image = np.transpose(image, (1,2,0))
#   mask = np.transpose(mask, (1,2,0))
#   print(np.max(image), np.max(mask))
#   visualize(
#       image=image.squeeze(),
#       # image=image, 
#       mask=mask[..., 0].squeeze(),
#   )
# sys.exit(1)

train_loader = torch.utils.data.DataLoader(
      train_dataset,
      batch_size=16,
      shuffle=True,
      pin_memory=True,
      drop_last=True)
total_step = len(train_loader)


data_path = '/content/Kvasir_fold_new/' + 'fold_' + str(i)
X_test = glob('{}/images/*'.format(data_path))
X_test.sort()
y_test = glob('{}/masks/*'.format(data_path))
y_test.sort()
test_dataset = Dataset_test(X_test, y_test,aug=False)
test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=1,
    shuffle=False,
    pin_memory=True,
    drop_last=True)

# ---- flops and params ----
params = model.parameters()
optimizer = torch.optim.Adam(params, lr)

Log.info("#"*20 + f"Start Training Fold{i}" + "#"*20)
print("#"*20, f"Start Training Fold{i}", "#"*20)
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

for epoch in range(start_from, 99):
    adjust_lr(optimizer, lr, epoch, decay_rate, decay_epoch)
    train(train_loader, test_loader , model, optimizer, epoch, test_fold,writer)

writer.flush()
writer.close()
end = timeit.default_timer()

Log.info("Training cost: "+ str(end - start) + 'seconds')

